<a href="https://colab.research.google.com/github/rishavsen1/gtfs_filtering/blob/main/trips_spd_lon_lat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip bustime_chattanooga_jan_2022.json.zip

Archive:  bustime_chattanooga_jan_2022.json.zip
  inflating: bustime_chattanooga_jan_2022.json  


In [2]:
import json
with open("bustime_chattanooga_jan_2022.json", "r") as f:
    data = json.load(f)

dated = []
for record in data:
    tmstmp = record["tmstmp"]
    if "20220111" in tmstmp:

        dated.append(record)



In [18]:
dated[1]

{'_id': {'$oid': '61dd0f014a63d75fa69c0601'},
 'des': None,
 'dly': 'false',
 'geometry': {'coordinates': ['-85.269788', '35.05660966666667'],
  'type': 'Point'},
 'hdg': '119',
 'lat': '35.05660966666667',
 'lon': '-85.269788',
 'mode': '0',
 'oid': 'N/A',
 'or': 'true',
 'origtatripno': None,
 'pdist': '0',
 'pid': '-1',
 'psgld': 'EMPTY',
 'rid': 'N/A',
 'rt': 'U',
 'spd': '1',
 'srvtmstmp': '20220111 00:00',
 'tablockid': 'N/A',
 'tatripid': 'N/A',
 'timestamp': 1641877200,
 'tmstmp': '20220111 00:00',
 'tripdyn': '0',
 'vid': '127',
 'zone': None}

In [15]:
# from shapely import geometry
[val.geometry.coordinates for val in dated]

AttributeError: ignored

In [21]:

for val in dated:
  val['coords'] = val['geometry']['coordinates']

In [83]:
dated[100]

{'_id': {'$oid': '61dd113e4a63d75fa69c0677'},
 'coords': ['-85.30130767822266', '35.051998138427734'],
 'des': 'Avondale',
 'dly': 'false',
 'geometry': {'coordinates': ['-85.30130767822266', '35.051998138427734'],
  'type': 'Point'},
 'hdg': '295',
 'lat': '35.051998138427734',
 'lon': '-85.30130767822266',
 'mode': '0',
 'oid': '160951',
 'or': 'false',
 'origtatripno': '205087',
 'pdist': '25276',
 'pid': '975',
 'psgld': 'EMPTY',
 'rid': '45',
 'rt': '10A',
 'spd': '0',
 'srvtmstmp': '20220111 00:10',
 'tablockid': '1551',
 'tatripid': '205087',
 'timestamp': 1641877800,
 'tmstmp': '20220111 00:10',
 'tripdyn': '0',
 'vid': '150',
 'zone': None}

In [84]:
import pandas as pd
import geopandas as gpd
# dated[10]
df = pd.DataFrame(dated)
df = df[['origtatripno', 'vid', 'tmstmp', 'timestamp', 'spd', 'lat', 'lon' ]]
# print(df.head())
df = df[df['timestamp']>1641890000]
df = df.dropna()
df = df.sort_values(by=['origtatripno', 'timestamp']).reset_index().drop(['index'], axis=1)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat'])).set_crs('EPSG:4326')

In [85]:
p1 = gdf[gdf['origtatripno'] == '204888']
# p1.plot()
gdf['origtatripno'].value_counts()

206125    203
206343    200
205923    190
204898    189
206312    188
         ... 
204959      6
205369      2
206037      2
205087      1
206048      1
Name: origtatripno, Length: 712, dtype: int64

In [89]:
gdf.to_csv('20220111_spd_lat_long.geojson')

In [90]:
from google.colab import files
files.download('20220111_spd_lat_long.geojson') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from datetime import time
data =dated
data_gen = []
vid = []
prev_trip_id = None
for record in data:
    trip_id = record["origtatripno"]
    if prev_trip_id != trip_id:
        vid = record['vid']
        srvtmstmp = record['srvtmstmp']
        data_gen.append({"trip_id": trip_id, "vid":vid, "start_time":srvtmstmp})
        prev_trip_id = trip_id

# unique_trp = list(set(data_gen))
# unique_trp.sort()
# data_gen
# print(len(unique_trp))

In [ ]:
len(data_gen)

44189

In [ ]:
import pandas as pd
data_gen_df = pd.DataFrame(data_gen, columns=["trip_id", 'vid', 'start_time'])
data_gen_df

,trip_id,vid,start_time
0,204959,112,20220111 00:00
1,None,127,20220111 00:00
2,204959,112,20220111 00:00
3,206345,152,20220111 00:00
4,205087,150,20220111 00:00
...,...,...,...
44184,None,150,20220111 23:59
44185,204959,126,20220111 23:59
44186,206345,111,20220111 23:59
44187,205087,154,20220111 23:59


In [ ]:
df_mod = data_gen_df.drop_duplicates(subset=["trip_id", "vid"]).reset_index(inplace=False).dropna()
df_mod["trip_id"] = df_mod["trip_id"]+"020"
df_mod = df_mod.drop(['index'], axis=1)
# df_mod = df_mod[["trip_id", "vid", 'start_time']]
df_mod

,trip_id,vid,start_time
0,204959020,112,20220111 00:00
2,206345020,152,20220111 00:00
3,205087020,150,20220111 00:00
6,206311020,504,20220111 04:21
9,206129020,113,20220111 04:39
...,...,...,...
759,206345020,111,20220111 23:25
760,205173020,154,20220111 23:25
761,204923020,126,20220111 23:37
762,204959020,126,20220111 23:57


In [ ]:
df_mod.to_csv('trips_vid_n_time_20220111.csv', index=False)

In [ ]:
import pandas as pd
import json
df = pd.read_csv("trips.txt", sep=",")

trips = unique_trp
trips = list(map(int, trips))

df_fil  = pd.DataFrame(columns = df.columns)

indices = []
for trip in trips:
    for index, row in df.iterrows():       
        if trip == int(row['trip_id']/1000):
            
            indices.append(index)
            df_fil.append(row, ignore_index=True)
            break


df_fil = df.iloc[indices]
print(df_fil.head())
df_fil.to_csv('df_fil_19.csv', index=False)


     trip_id route_id  service_id trip_headsign  trip_short_name  \
0  204888020        1           1    ALTON PARK              NaN   
1  204889020        1           1    ALTON PARK              NaN   
2  204890020        1           1    ALTON PARK              NaN   
3  204891020        1           1    ALTON PARK              NaN   
4  204892020        1           1    ALTON PARK              NaN   

   direction_id  block_id  shape_id  wheelchair_accessible  bikes_allowed  \
0             0      1002  shp-1-04                      2              2   
1             0      2802  shp-1-03                      2              2   
2             0      3002  shp-1-03                      2              2   
3             0      2802  shp-1-03                      2              2   
4             0      3002  shp-1-03                      2              2   

   block_name  
0         107  
1         155  
2         156  
3         155  
4         156  
